In [1]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, IQPAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
import utilities 

/home/adriano22_/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from discopy.tensor import Tensor
from sympy import default_sort_key
from jax import jit, grad
import numpy as np
import warnings

In [3]:
bobcat_parser = BobcatParser(verbose = "progress")
tensor_ansatz = TensorAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2), AtomicType.CONJUNCTION: Dim(2), AtomicType.PUNCTUATION: Dim(2), AtomicType.NOUN_PHRASE: Dim(2), AtomicType.PREPOSITIONAL_PHRASE: Dim(2)})

In [ ]:
pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\ePurse_edited.csv", "n")
#train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\tutorial\\mc_train_data.txt", "n")
#test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\tutorial\\mc_test_data.txt", "n")

In [5]:
#linux
pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

Tagging sentences:   0%|          | 0/42 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/168 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/168 [00:00<?, ?it/s]

Tagging sentences:   0%|          | 0/31 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/124 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/124 [00:00<?, ?it/s]

In [6]:
warnings.filterwarnings('ignore')
np.random.seed(123458) 

#creation of the vocabulary
circuits = train_circuits + test_circuits
vocab = sorted({sym for circ in circuits for sym in circ.free_symbols}, key=default_sort_key)
tensors = [np.random.rand(w.size) for w in vocab]

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def loss(tensors):
    np_circuits = [c.lambdify(*vocab)(*tensors) for c in train_circuits]
    predictions =  sigmoid(np.array([c.eval().array for c in np_circuits]))
    cost = -np.sum(train_labels * np.log2(predictions)) / len(train_labels)
    return cost

In [8]:
#training
training_losses = []
epochs = 70
for i in range(epochs):
    gr = loss(tensors)
    for k in range(len(tensors)):
        tensors[k] = tensors[k] - gr * 1.0

    training_losses.append(float(loss(tensors)))
    if (i + 1) % 10 == 0: print(f"Epoch {i + 1} - loss {training_losses[-1]}")

: 

: 

In [13]:
#evaluation
np_test_circuits = [c.lambdify(*vocab)(*tensors) for c in test_circuits]
test_predictions =  sigmoid(np.array([c.eval().array for c in np_test_circuits]))
print("ciao")

hits = 0
for i in range(len(np_test_circuits)):
    target = test_labels[i]
    pred = test_predictions[i]
    if np.argmax(target) == np.argmax(pred):
        hits += 1

print("Accuracy on test set:", hits / len(np_test_circuits))